# 패키지 설치
pip 명령어로 의존성 있는 패키지를 설치합니다.



In [1]:
!pip install ratsnlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 102.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.1
    Uninstalling tokenizers-0.15.1:
      Successfully uninstalled tokenizers-0.15.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


# 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

In [3]:
from ratsnlp.nlpbook.ner import NERDeployArguments
args = NERDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-ner",
    max_seq_length=64,
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-ner/epoch=1-val_loss=0.11.ckpt
downstream_model_labelmap_fpath: /gdrive/My Drive/nlpbook/checkpoint-ner/label_map.txt


# 모델 로딩
파인튜닝을 마친 모델과 토크나이저를 읽어 들입니다.

In [4]:
import torch
from transformers import BertConfig, BertForTokenClassification
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)
model = BertForTokenClassification(pretrained_model_config)
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [5]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

# 레이블 맵 작성

범주 인덱스를 범주명과 매칭하는 사전을 만듭니다.
확률값의 각 요소값이 어떤 태그에 대응하는지 정보를 알고 있어야 합니다.
이와 관련해 다음 코드를 실행하면 정수 인덱스를 레이블에 매핑하는 사전을 만듭니다.

In [6]:
labels = [label.strip() for label in open(args.downstream_model_labelmap_fpath, "r").readlines()]
id_to_label = {}
for idx, label in enumerate(labels):
  if "PER" in label:
    label = "인명"
  elif "LOC" in label:
    label = "지명"
  elif "ORG" in label:
    label = "기관명"
  elif "DAT" in label:
    label = "날짜"
  elif "TIM" in label:
    label = "시간"
  elif "DUR" in label:
    label = "기간"
  elif "MNY" in label:
    label = "통화"
  elif "PNT" in label:
    label = "비율"
  elif "NOH" in label:
    label = "기타 수량표현"
  elif "POH" in label:
    label = "기타"
  else:
    label = label
  id_to_label[idx] = label

In [7]:
print(id_to_label)

{0: '[CLS]', 1: '[SEP]', 2: '[PAD]', 3: '[MASK]', 4: 'O', 5: '인명', 6: '기타 수량표현', 7: '기타', 8: '기관명', 9: '날짜', 10: '지명', 11: '통화', 12: '비율', 13: '시간', 14: '기간', 15: '인명', 16: '기타 수량표현', 17: '기타', 18: '기관명', 19: '날짜', 20: '지명', 21: '통화', 22: '비율', 23: '시간', 24: '기간'}


# 인퍼런스 함수 선언
인퍼런스 함수를 선언합니다.

In [8]:
def inference_fn(sentence):
  # 문장을 토큰화하고 인덱싱하되,
  # max_seq_length보다 짧으면 맞게 패딩하고 길면 자르기
  inputs = tokenizer(
      [sentence],
      max_length=args.max_seq_length,
      padding="max_length",
      truncation=True,
  )
  with torch.no_grad():
    # 모델 계산하기
    outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
    # inputs를 파이토치 텐서로 변환하기
    # 로짓에 소프트맥스를 취해 각 토큰이 어떤 개체명에 속하는지 확률 구하기
    probs = outputs.logits[0].softmax(dim=1)
    top_probs, preds = torch.topk(probs, dim=1, k=1)
    # 각 토큰이 속하는 개체명 확률 분포 가운데(probs)
    # 가장 높은 확률 값(top_probs)과 그에 속하는 개체명 인덱스(preds) 구하기
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    # 토큰 인덱스 시퀀스(List[int])를 토큰 시퀀스로 변환하기
    predicted_tags = [id_to_label[pred.item()] for pred in preds]
    # 개체명 인덱스 시퀀스(List[int])를 개체명 시퀀스로 변환하기
    result = []
    # [CLS], [SEP], [PAD]를 제외한 토큰 각각에 대해 모델이 예측한 개체명과(predicted_tag)과
    # 그 확률값(top_prob)을 반환하기
    for token, predicted_tag, top_prob in zip(tokens, preicted_tags, top_probs):
      if token not in [tokenizer.pad_token, tokenizer.cls_token, tokenizer.sep_token]:
        token_result = {
            "token": token,
            "predicted_tag": predicted_tag,
            "top_prob": str(round(top_prob[0].item(), 4)),
        }
        result.append(token_result)

    return {
        "sentence": sentence,
        "result": result,
    }

# 웹서비스 만들기 준비

`ngrok`은 코랩 로컬에서 실행 중인 웹서비스를 안전하게 외부에서 접근 가능하도록 해주는 도구입니다. `ngrok`을 실행하려면 [회원가입](https://dashboard.ngrok.com/signup) 후 [로그인](https://dashboard.ngrok.com/login)을 한 뒤 [이곳](https://dashboard.ngrok.com/get-started/your-authtoken)에 접속해 인증 토큰(authtoken)을 확인해야 합니다. 예를 들어 확인된 `authtoken`이 `test111`이라면 다음과 같이 실행합니다.

```bash
!mkdir /root/.ngrok2 && echo "authtoken: test111" > /root/.ngrok2/ngrok.yml
```

In [10]:
!mkdir /root/.ngrok2 && echo "authtoken: {이곳에 확인된 인증 토큰을 입력하세요}" > /root/.ngrok2/ngrok.yml

In [ ]:
from ratsnlp.nlpbook.ner import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()